# 目标
爬取苦瓜书盘的数据，并下载图书文件。
支持单线程与多线程爬虫

![](http://881023.top/image/爬书战果.png)

In [1]:
import os
from requests_html import HTMLSession
import urllib
import urllib.request
import requests
# https
import ssl
ssl._create_default_https_context=ssl._create_unverified_context
# 线程池
from concurrent.futures import ThreadPoolExecutor,wait,as_completed

In [6]:
session = HTMLSession()
# 创建一个ua，绕过初级反爬
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0'
headers = {'User-Agent': ua}
# 创建一个最大数量为3的线程池
executor = ThreadPoolExecutor(max_workers=3)
# 跳过前几页
firstPage = 50

In [3]:
# 获取页面最后一页的地址,str
def get_last_page_url():
    # 返回一个 response 对象
    response = session.get('https://kgbook.com/list/index.html', headers=headers)
    pages = response.html.find('div.pagenavi', first=True)
    a_list = pages.find('a')
    urlObject = a_list[len(a_list) - 1]
    u = urlObject.absolute_links
    ustr = ''.join(u)
    return ustr;

# 获取页的数量
def get_total_pages(ustr):
    back = ustr.split('index_')[1]
    num = back.split('.')[0]
    print("一共%s页" % num)
    return int(num);
    
# 创建页地址数组
def get_total_page_urls(num):
    urls = []
    for index in range(num):
        if index <= 0:
            urls.append('https://kgbook.com/list/index.html')
        else:
            urls.append('https://kgbook.com/list/index_'+ str(index+1) +'.html')
    return urls
    
# 获取当前页面数据
def get_books_in_page(pageUrl, num):
    print('开始分析page', str(num+1), '>>>>>>>')
    s = session.get(pageUrl, headers=headers)
    books = s.html.find('div.channel-item a')
    #a_list = books.find('a')
    return books
    
# 根据一个bookdetailUrl下载书籍
def download_bookfile(pageNum, bookDetailUrl, bookname):
    print('开始解析下载', bookname, '当前页数', str(pageNum+1), '->->->')
    s = session.get(bookDetailUrl, headers=headers)
    btns = s.html.find('a.button')
    for btn in btns:
        bookurl = ''.join(btn.absolute_links)
        # 如果是下载链接
        if 'GetDown' in bookurl:
            realUrl, extendName = get_real_url(bookurl)
            bkname = bookname + '.' + extendName
            # 判断是不是已经存在了这本书
            if is_exist_book(bkname):
                print('\033[1;31m %s 已存在，跳过下载。 \033[0m' % bkname)
                continue
            downloadfile(realUrl, bkname)
    print(bookname, '下载完毕')
    print('-----------------------------')

# 下载文件
def downloadfile(url, filename):
    print('开始下载文件')
    try:
        # r = requests.get(url)
        path = "books/"+filename
        r = requests.get(url, headers=headers)
        with open(path,"wb") as f:
            f.write(r.content)
        f.close()
    except Exception:
        print('\033[1;31m 下载失败 \033[0m')
    else:
        print("\033[1;32m 下载", filename, "完成\033[0m")
    
# 获取重定向真实url
def get_real_url(url):
    print('开始获取真实下载地址')
    # http_headers = { 'Accept': '*/*','Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'}
    rs = requests.get(url,headers=headers)
    print(rs.url)
    # 获取文件的拓展名
    words = rs.url.split('.')
    extendName = words[len(words) - 1]
    print('这是一个', extendName, '文件')
    return rs.url, extendName

# 目录下是否存在这本书
def is_exist_book(bookname):
    if os.path.exists('books/'+bookname):
        return True
    else:
        return False


## 单线程进行书籍爬取

In [ ]:
# 全部最新更新书籍
url = get_last_page_url()
num = get_total_pages(url)
urls = get_total_page_urls(num)
# 遍历page
for index in range(len(urls)):
    #调整起始页
    if index < firstPage:
        print('跳过第%s页' % str(index+1))
        continue
    onepage = urls[index]
    bookset = get_books_in_page(onepage, index)
    # 遍历单页书籍
    for book in bookset:
        # 属性集合
        attrs = book.attrs
        # 如果属性里面没有title字段，说明不是一个书籍详情的链接
        if not 'title' in attrs.keys():
            continue
        title = book.text
        bookDetailUrl = ''.join(book.absolute_links)
        download_bookfile(index, bookDetailUrl, title)

## 多线程进行数据爬取

In [ ]:
# 全部最新更新书籍
url = get_last_page_url()
num = get_total_pages(url)
urls = get_total_page_urls(num)
# 遍历page
for index in range(len(urls)):
    #调整起始页
    if index < firstPage:
        print('跳过第%s页' % str(index+1))
        continue
    onepage = urls[index]
    bookset = get_books_in_page(onepage, index)
    futures = []
    # 遍历单页书籍
    for book in bookset:
        # 属性集合
        attrs = book.attrs
        # 如果属性里面没有title字段，说明不是一个书籍详情的链接
        if not 'title' in attrs.keys():
            continue
        title = book.text
        bookDetailUrl = ''.join(book.absolute_links)
        # 把任务加入线程池队列
        print('把%s加入任务队列' % title)
        future = executor.submit(download_bookfile,index,bookDetailUrl,title)


开始获取真实下载地址
 下载 风起陇西.pdf 完成
风起陇西 下载完毕
-----------------------------
开始解析下载 飞狐外传 当前页数 59 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201012/c939bce2a412115f3b3c0399559a0441.pdf
这是一个 pdf 文件
开始下载文件
 下载 连城诀.pdf 完成
连城诀 下载完毕
-----------------------------
开始解析下载 碧血剑 当前页数 59 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201012/9c214ce7ddb646e849c4f7e6a15078fe.pdf
这是一个 pdf 文件
开始下载文件
https://kgbook.com/d/file/201012/2ca19ee3e5d35e4b06752d99c158c63b.pdf
这是一个 pdf 文件
开始下载文件
 下载 飞狐外传.pdf 完成
飞狐外传 下载完毕
-----------------------------
开始解析下载 闲看水浒：字缝里的梁山规则与江湖世界 当前页数 59 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201012/53d2293660f3d3e5ca2a5f226b0784e6.pdf
这是一个 pdf 文件
开始下载文件
 下载 笑傲江湖.pdf 完成
笑傲江湖 下载完毕
-----------------------------
开始解析下载 古代汉语(第四册) 当前页数 59 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201012/571bac77da873dcde8eea341836b1fbf.pdf
这是一个 pdf 文件
开始下载文件
 下载 闲看水浒：字缝里的梁山规则与江湖世界.pdf 完成
闲看水浒：字缝里的梁山规则与江湖世界 下载完毕
-----------------------------
开始解析下载 儿歌简笔画 当前页数 59 ->->->
开始获取真实下载地址
https://kgbook.com/d/file/201

In [ ]:
executor.shutdown()